# Personalized Movie Recommendations
This notebook shows how to build a personalized movie recommendation model with ThirdAI's Universal Deep Transformer (UDT) model, our all-purpose classifier for tabular datasets. In this demo, we will train and evaluate the model on the Movielens 1M dataset, but you can easily replace this with your own dataset.

To run this notebook, you will need to obtain a ThirdAI license at the following link if you have not already: https://www.thirdai.com/try-bolt/

In [ ]:
!pip3 install thirdai
!pip3 install pandas # We need Pandas in the utils module to convert downloaded datasets to CSV format
!pip3 install numpy # We use numpy to analyze UDT performance in this notebook

# Dataset Download
We will use the utils module in this repo to download dataset_name (if you have just copied this notebook and not cloned the entire repo, you will need to copy the utils.py file as well). You can replace this step and the next step with a download method and a UDT initialization step that is specific to your dataset.

In [ ]:
import utils
train_filename, test_filename, inference_batch, index_batch = utils.download_movielens()

# UDT Initialization
We can now create a UDT model by passing in the types of each column in the dataset and the target column we want to be able to predict.

For this demo, we additionally want to use "temporal context" to make predictions. Adding temporal context requires a single bolt.types.date() column to use to track the timestamp of training data. We pass in a dictionary called temporal_tracking_relationships that tells UDT we want to track movies over time for each user. This allows UDT to make better predictions for the target column by creating temporal features that take into account the historical relationship between users and movies.

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "userId": bolt.types.categorical(n_unique_classes=6040),
        "movieTitle": bolt.types.categorical(n_unique_classes=3706),
        "timestamp": bolt.types.date(),
    },
    temporal_tracking_relationships={
        "userId": ["movieTitle"]
    },
    target="movieTitle",
)

# Training
We can now train our UDT model with just two lines! Feel free to customize the number of epochs and the learning rate; we have chosen values that give good convergence.

In [ ]:
train_config = (bolt.TrainConfig(epochs=3, learning_rate=0.001)
                    .with_metrics(["recall@10"]))

model.train(train_filename, train_config);

# Evaluation
Evaluating the performance of the UDT model is also just two lines!

In [ ]:
eval_config = (bolt.EvalConfig()
                   .with_metrics(["recall@1", "recall@10", "recall@100"]))

model.evaluate(test_filename, eval_config);

# Saving and Loading
Saving and loading a trained UDT model to disk is also extremely straight forward.

In [ ]:
save_location = "personalized_movie_recommendation.model"

# Saving
model.save(save_location)

# Loading
model = bolt.UniversalDeepTransformer.load(save_location)

# Testing Predictions
The evaluation method is great for testing, but it requires labels, which don't exist in a production setting. We also have a predict method that can take in an in-memory batch of rows or a single row (without the target column), allowing easy integration into production pipelines.

In [ ]:
print(inference_batch)

In [ ]:
import numpy as np

print("Inference batch:", inference_batch)

prediction = model.predict(inference_batch[0])
class_name = model.class_name(np.argmax(prediction))
print("Input:", inference_batch[0], "Prediction:", class_name, "\n")

prediction_batch = model.predict_batch(inference_batch)
class_names = [model.class_name(class_id) for class_id in np.argmax(prediction_batch, axis=1)]
print("Batch Prediction Results")
for input_sample, class_name in zip(inference_batch, class_names):
    print("Input:", input_sample, "Prediction:", class_name)

You may notice that we keep recommending the same movies to the same person. This is because we don't update UDT's temporal trackers with new information. If we "index" new observations as we get them, then UDT will take advantage of this new information to make better predictions. When you run the following cell, notice how the prediction changes in response to new data.

In [ ]:
print("Observations batch:", inference_batch)

# Let's reset the temporal trackers whenever we run this cell to keep results consistent.
# This clears the watch histories of every user. 
model.reset_temporal_trackers()

for input_sample, new_observation in zip(inference_batch, index_batch):
    prediction = model.predict(input_sample)
    top_10_ids = np.argsort(-prediction)[:10]
    top_10_class_names = [model.class_name(class_id) for class_id in top_10_ids]
    print("Single Prediction Result")
    print("Input:", input_sample, "Predictions:", top_10_class_names)
    
    # Index new observations as we get them to help UDT make informed predictions.
    model.index(new_observation)
    print("Indexed ground truth sample:", new_observation, "\n")

# We can also index an entire batch of observations.
model.index_batch(index_batch)